<a href="https://colab.research.google.com/github/staniya/shibaki-RF-analysis/blob/main/BN_RSver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RFモデルを使ってAE-FILD_BAL_AI解析を行う

### 必要ライブラリの導入と環境設定

In [1]:
import pandas as pd
import numpy as np
# データの図を出力するライブラリ
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# 環境設定
import sys
sys.path.append('models')

# 実際の機械学習ライブラリ
import tensorflow as tf

In [2]:
# Google Driveからデータを引っ張ってくる
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/shibaki analysis/AE-FILD BAL patient_data.xlsx' patient_data.xlsx

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### データ処理

In [3]:
!python -m pip install openpyxl==3.0
# openpyxlというエンジンを使用して、ExcelファイルをPandasのデータフレームとして読みこむ
AE_FILD_df = pd.read_excel('patient_data.xlsx', engine='openpyxl')

In [4]:
# ヘルパーファンクションでテキスト内の"\n"をスペースに変換する関数を定義する
def clean_header(text):
  text = text.strip() # 最初と最後のスペースをテキストからとる
  text = text.replace(" ", "-")
  text_n_newline = text.replace("\n", " ")
  return text_n_newline

In [5]:
# ヘッダーコラムのテキストをリストとして変換する
AE_FILD_header_list = AE_FILD_df.columns.values.tolist()

# 上記のヘッダーリストにclean_header()を応用する
AE_FILD_header_list_cleaned = [clean_header(x) for x in AE_FILD_header_list]
print(AE_FILD_header_list_cleaned)

AE_FILD_df.columns = AE_FILD_header_list_cleaned
AE_FILD_df.head()

['No', 'ID', '90day-mortality survival:0 death:1', 'sex male:0 female:1', 'age', 'smoking-status Never:0 Ex(quit-smoking-over-a-year-ago):1 Current:2', 'Brinkman-Index', 'emphysema (-):0 (+):1', 'Baseline-ILD UIP:0 probable-UIP:1 Indeterminate-for-UIP:2 Alternative-diagnosis:3', 'Collagen-disease (-):0 (+):1', 'IPAF (-):0 (+):1', 'P/F-ratio', 'LDH', 'CRP', 'KL-6', 'BAL(neutro-%)', 'BAL(Lymph-%)', 'BAL(Eos-%)', 'BAL(M-%)', 'BAL(CD4/8)', 'MMP-1', 'MMP-7', 'TGFB1', 'TGFB2', 'TGFB3', 'EGF', 'FGF-2', 'EOTAXIN', 'TGF-a', 'G-CSF', 'Flt-3L', 'GM-CSF', 'FRACTALKINE', 'IFN-a2', 'IFN-g', 'GRO', 'IL-10', 'MCP-3', 'IL-12P40', 'MDC', 'IL-12P70', 'IL-13', 'IL-15', 'sCD40L', 'IL-17', 'IL-1RA', 'IL-1a', 'IL-9', 'IL-1b', 'IL-2', 'IL-3', 'IL-4', 'IL-5', 'IL-6', 'IL-7', 'IL-8', 'IP-10', 'MCP-1', 'MIP-1a', 'MIP-1b', 'TNFa', 'TNFb', 'VEGF', 'IL-36β']


,No,ID,90day-mortality survival:0 death:1,sex male:0 female:1,age,smoking-status Never:0 Ex(quit-smoking-over-a-year-ago):1 Current:2,Brinkman-Index,emphysema (-):0 (+):1,Baseline-ILD UIP:0 probable-UIP:1 Indeterminate-for-UIP:2 Alternative-diagnosis:3,Collagen-disease (-):0 (+):1,IPAF (-):0 (+):1,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,K003,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,K022,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,K024,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,K043,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,K045,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [6]:
# まず機械学習に不必要なコラムをdrop()する
AE_FILD_df = AE_FILD_df.drop(columns=["No"])
AE_FILD_df.head()

,ID,90day-mortality survival:0 death:1,sex male:0 female:1,age,smoking-status Never:0 Ex(quit-smoking-over-a-year-ago):1 Current:2,Brinkman-Index,emphysema (-):0 (+):1,Baseline-ILD UIP:0 probable-UIP:1 Indeterminate-for-UIP:2 Alternative-diagnosis:3,Collagen-disease (-):0 (+):1,IPAF (-):0 (+):1,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


#### データセット内の欠測値(NaN)処理
sklearnのSimpleImputerを使うhttps://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

In [7]:
# データセット内のNaN値をどうするか (まずはinputer transformで試してみる)
from sklearn.impute import SimpleImputer
AE_FILD_df.replace('NaN', np.NaN, inplace=True) # データフレーム内の"NaN"をnumpyのNaNと置き換える
imp = SimpleImputer(missing_values=np.NaN, strategy="median") # sklearnのSimpleImputerを使い、np.NaNをnull値として認識させる。加、strategy="median"はnp.NaNをその行のNaNを中央値と置き換えるための指定
AE_FILD_df_idf = pd.DataFrame(imp.fit_transform(AE_FILD_df))
AE_FILD_df_idf.columns = AE_FILD_df.columns # SimpleImputerは数値しか作成できない為、元のデータセットの行名を新しいデータセットに加るコード
df_null_val = list(AE_FILD_df_idf.isna().sum()) # データセット内にNaN値が無いかの確認
print(f"AE_FILD_df_idfデータフレーム内のNaN数：{df_null_val}")

AE_FILD_df_idfデータフレーム内のNaN数：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### データセット内のカテゴリカル変数の数値をその数値が表すカテゴリーに置き換える必要がある際は下記の3つのせるを実行する

In [8]:
from collections import OrderedDict

columnname_dict = OrderedDict()
columnname_dict['90day-mortality'] = {"survival":0.0, "death":1.0}
columnname_dict['sex'] = {"male":0.0, "female":1.0}
columnname_dict['smoking-status'] = {"Never":0.0, 'Ex(quit smoking over a year ago)':1.0, "Current":2.0}
columnname_dict['emphysema'] = {"-":0.0, "+":1.0}
columnname_dict['Baseline-ILD'] =  {"UIP":0.0, "probable UIP":1.0, "Indeterminate for UIP":2.0, "Alternative diagnosis":3.0}
columnname_dict['Collagen-disease'] = {"-":0.0, "+":1.0}
columnname_dict['IPAF'] = {"-":0.0, "+":1.0}

for key,values in columnname_dict.items():
  temp_dict = {}
  for k,v in values.items():
    temp_dict[v] = k
    columnname_dict[key] = temp_dict

columnname_dict

OrderedDict([('90day-mortality', {0.0: 'survival', 1.0: 'death'}),
             ('sex', {0.0: 'male', 1.0: 'female'}),
             ('smoking-status',
              {0.0: 'Never',
               1.0: 'Ex(quit smoking over a year ago)',
               2.0: 'Current'}),
             ('emphysema', {0.0: '-', 1.0: '+'}),
             ('Baseline-ILD',
              {0.0: 'UIP',
               1.0: 'probable UIP',
               2.0: 'Indeterminate for UIP',
               3.0: 'Alternative diagnosis'}),
             ('Collagen-disease', {0.0: '-', 1.0: '+'}),
             ('IPAF', {0.0: '-', 1.0: '+'})])

In [9]:
# ヘルパーファンクションでテキスト内の記号と置き換える
def remove_header_map(text):
  for key in columnname_dict.keys():
    if key in text:
      text = key
  return text

# ヘッダーコラムのテキストをリストとして変換する
AE_FILD_header_list = AE_FILD_df_idf.columns.values.tolist()
AE_FILD_header_list_cleaned = [remove_header_map(x) for x in AE_FILD_header_list]
AE_FILD_df_idf.columns = AE_FILD_header_list_cleaned
AE_FILD_df_idf.head()

,ID,90day-mortality,sex,age,smoking-status,Brinkman-Index,emphysema,Baseline-ILD,Collagen-disease,IPAF,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,558277.0,0.0,0.0,78.0,1.0,1600.0,0.0,1.0,0.0,0.0,291.666667,345.0,2.35,1599.0,41.0,38.0,3.0,13.0,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,6.246947,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,1848827.0,0.0,1.0,72.0,0.0,0.0,0.0,2.0,0.0,0.0,264.571429,277.0,5.65,1620.0,18.0,21.0,0.0,61.0,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,6.246947,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,1902349.0,0.0,0.0,86.0,0.0,0.0,0.0,2.0,0.0,0.0,250.000000,269.0,6.50,1073.0,2.0,81.0,4.0,12.0,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,6.246947,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,3165531.0,0.0,0.0,58.0,1.0,500.0,0.0,2.0,0.0,0.0,200.000000,538.0,11.26,2392.0,13.0,47.0,14.0,25.0,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,6.246947,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,3.2,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,3.792897,3.2,3.2,2.000000
4,3253683.0,0.0,0.0,76.0,1.0,800.0,0.0,3.0,0.0,0.0,380.952381,372.0,6.43,1555.0,9.0,37.0,36.0,7.0,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,4.350069,6.246947,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [10]:
# for columns in AE_FILD_df_idf.columns:
#     for key,value in columnname_dict.items():
#       if columns == key:
#         AE_FILD_df_idf[columns].replace(value,inplace=True)
# AE_FILD_df_idf.head()

#### 機械学習を行う為の学習データが不足している場合、DataSynthesizerライブラリーを導入し模擬データを作成する
リンク：https://github.com/DataResponsibly/DataSynthesizer

ベイズ法を応用した模擬データの作成方法についてはこの論文を参照してください：https://www.scb.se/contentassets/ca21efb41fee47d293bbee5bf7be7fb3/using-bayesian-networks-to-create-synthetic-data.pdf

In [11]:
# 模擬データを作るため、DataSynthesizer ライブラリーの導入
!pip install -q DataSynthesizer
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

In [12]:
import os
# 上で作成したPandasのデータフレームをDataSynthesizerが応用できるようにcsvに変換する
AE_FILD_df_idf.to_csv("AE_FILD_df_idf.csv", index=False)

# 模擬データが元のデータの類似するように模擬データの作成設定をcorrelated_attribute_modeにする
mode = 'correlated_attribute_mode'

# 模擬データの構造を説明するための情報ファイル等の指定
if not os.path.isdir('./out'):
  os.makedirs("./out") # 出力ディレクトリーの作成
if not os.path.isdir(f"./out/{mode}"):
  os.makedirs(f"./out/{mode}") # 模擬データのモード名を元にサブ出力ディレクトリーの作成
description_file = f'./out/{mode}/description.json'
synthetic_data = f'./out/{mode}/synthetic_data.csv'

In [13]:
categorical_attributes = {}

# データフレーム内の0−9行の内age以外の行はカテゴリカルなため、模擬データを作成出来るようにカテゴリかルアトリビュートとして認識する
for columns in list(AE_FILD_df_idf.columns[:9]):
  if columns != 'ID' and columns != 'age'and columns != 'Brinkman-Index':
    categorical_attributes[columns] = True
# データの要素を識別できるように、IDをデータのキーとして認識する
candidate_keys = {'ID': True}

# OrderedDictがちゃんと作成されているか確認するコード
for key, val in categorical_attributes.items():
  print(f"{key}: {val}")

90day-mortality: True
sex: True
smoking-status: True
emphysema: True
Baseline-ILD: True
Collagen-disease: True


In [14]:
# # ドメインサイズがこのしきい値よりも小さい場合、属性はカテゴリーに分類されます。
# # 今回のデータでは一番多いカテゴリー要素が4だったので、閾値を4と指定する
# threshold_value = 4

# # 差分プライバシーのパラメーター。 これは、入力データセットの行を削除しても、exp（epsilon）の乗法差以上に同じ出力を取得する確率が変わらないことを大まかに意味します。 
# # イプシロン値を増やすことで学習モデルのノイズを減らせます。 差分プライバシーをオフにするにはepsilon = 0を設定します。
# epsilon = 1

# # ベイジアンネットワーク内の親ノードの最大数、つまり、有向グラフでの入力エッジの最大数。
# degree_of_bayesian_network = 2

# # DataDescriberをインスタンス化する
# describer = DataDescriber(category_threshold=threshold_value)

# # データセットの統計を計算する
# describer.describe_dataset_in_correlated_attribute_mode(dataset_file='./AE_FILD_df_idf.csv', 
#                                                         epsilon=epsilon, 
#                                                         k=degree_of_bayesian_network,
#                                                         attribute_to_is_categorical=categorical_attributes,
#                                                         attribute_to_is_candidate_key=candidate_keys)

In [15]:
# # データセットの統計を上で定義した出力ファイルに保存する
# # その後に作成されたベイズネットワークを出力する
# describer.save_dataset_description_to_file(description_file)
# display_bayesian_network(describer.bayesian_network)

In [16]:
# description_fileをグーグルドライブから引っ張ってくる
!cp '/content/drive/MyDrive/shibaki analysis/description.json' out/correlated_attribute_mode/description.json

# synthetic_dataをグーグルドライブから引っ張ってくる
!cp '/content/drive/MyDrive/shibaki analysis/synthetic_data.csv' out/correlated_attribute_mode/synthetic_data.csv

In [17]:
# # 合成データセットで生成される症例数。
# num_tuples_to_generate = 50

# # 実際に模擬データを作成するコード
# generator = DataGenerator()

# # ベイズモデルの応用（模擬データの作成）
# # description_fileとして定義してあるJSONファイルにベイズモデルが学習したカテゴリカルアトリビュートの特徴が保存される
# # num_tuples_to_generateは生成される症例数
# generator.generate_dataset_in_correlated_attribute_mode(num_tuples_to_generate, description_file)

# # 作成した模擬データをsynthetic_dataとして定義したcsvファイルに保存する
# generator.save_synthetic_data(synthetic_data)

# pandasのデータフレームとして模擬データを読み込む
synthetic_df = pd.read_csv(synthetic_data)
synthetic_df.head()

,ID,90day-mortality,sex,age,smoking-status,Brinkman-Index,emphysema,Baseline-ILD,Collagen-disease,IPAF,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,0,1.0,1.0,86.0,1.0,2394.0,1.0,3.0,0.0,1.0,407.570527,3738.0,24.316082,0,57.0,45.0,30.0,84.0,0.168003,2799.125375,48323.479249,108.723195,177.718030,20.432361,12.301286,9.634568,30.619083,3.200000,123.840457,6.889319,3.754871,49.394264,12.417987,18.578464,2811.101402,13.689079,41.860992,3.685562,627.991972,3.2,11.592210,3.229818,19.475090,3.2,1137.306709,45.017316,3.2,21.593532,3.2,3.2,14.548051,5.474744,74.543766,3.548324,694.838787,15478.751005,3078.607437,3.281641,547.590709,25.540368,3.2,25.515875,4.221958
1,1,1.0,0.0,60.0,2.0,206.0,0.0,2.0,0.0,1.0,151.077675,2590.0,22.684567,1,65.0,65.0,24.0,26.0,7.007804,1266.869152,45683.866004,15.190320,18.717371,18.211128,6.546231,20.203311,66.983223,3.200000,412.280494,7.428806,8.848896,31.967984,13.899633,8.992876,1391.136745,20.897231,151.059845,3.430148,686.099361,3.2,10.052091,4.298356,16.352019,3.2,408.765667,34.006108,3.2,10.858751,3.2,3.2,5.954314,27.780880,11.478718,6.628477,135.143375,5364.580494,4123.436242,36.210179,25.071161,27.191287,3.2,46.806025,4.221958
2,2,0.0,1.0,67.0,1.0,2719.0,1.0,0.0,0.0,1.0,259.370996,195.0,22.544898,2,50.0,4.0,26.0,80.0,10.157648,2595.779366,31808.574858,95.953388,91.602891,10.798302,4.103280,18.715837,44.172413,3.992006,89.403799,8.839550,6.557296,101.325366,14.397230,10.428198,2012.597003,35.726709,96.878026,4.414629,699.732207,3.2,6.502391,4.687136,7.363623,3.2,1201.528415,33.620035,3.2,5.683322,3.2,3.2,30.867292,23.424122,79.820294,4.980176,969.864985,13676.112447,3341.439541,157.990737,341.947937,28.024008,3.2,48.160894,4.221958
3,3,0.0,1.0,79.0,2.0,2326.0,0.0,2.0,0.0,1.0,269.170305,922.0,2.421177,3,62.0,43.0,29.0,55.0,5.405535,3586.037858,22452.338950,29.756084,95.042749,14.646034,4.805881,19.163974,66.926482,3.200000,380.012277,3.345372,7.688496,121.065591,13.845726,5.889702,1363.064285,9.923964,133.371213,3.220648,470.960500,3.2,3.599722,4.231428,9.524234,3.2,73.445888,38.900621,3.2,13.533619,3.2,3.2,12.156582,8.334745,386.159171,4.483727,660.711015,8288.883647,2897.711352,110.080796,239.129284,58.162618,3.2,46.039830,4.221958
4,4,1.0,1.0,80.0,1.0,642.0,0.0,3.0,1.0,0.0,116.411842,2767.0,14.745234,4,35.0,26.0,28.0,82.0,5.916806,2956.635533,9110.319452,74.380000,158.378141,17.269699,5.275159,7.582229,37.423971,3.992006,97.087884,8.531709,6.103819,24.832766,26.072135,5.975976,379.263151,35.036860,63.839941,4.131372,548.219353,3.2,4.968551,3.384449,14.701829,3.2,72.040653,44.732907,3.2,29.690933,3.2,3.2,38.804723,17.533635,455.553406,6.897578,385.929507,10971.900333,2244.424453,121.848466,340.248225,7.434939,3.2,34.543211,4.221958


In [18]:
# 新しく作成した模擬データは上記で作成したsynthetic_df内にあるため、AE_FILD_df_idfにデータを足す
AE_FILD_idf_syn = pd.concat([AE_FILD_df_idf, synthetic_df])

#### もし上記のセルでデータセット内のカテゴリカル変数の数値をその数値が表すカテゴリーに置き換えた場合、機械学習を行えるように下記のセルで数値に置き換える


In [19]:
# # ベイズネットワークを作成するためにデータの要素をストリングに変換したため、上記で作成したディクショナリーを元に数字に変換する
# for columns in AE_FILD_idf_syn.columns:
#     for key,value in columnname_dict.items():
#       if columns == key:
#         for sub_k, sub_v in value.items():
#           AE_FILD_idf_syn[columns].replace({sub_k:value}, inplace=True)
# AE_FILD_idf_syn.head()

### RF解析モデルの定義

In [20]:
#IDを削除
AE_FILD_idf_syn = AE_FILD_idf_syn.drop(labels=['ID'], axis=1)
AE_FILD_idf_syn.head(15)

,90day-mortality,sex,age,smoking-status,Brinkman-Index,emphysema,Baseline-ILD,Collagen-disease,IPAF,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,0.0,0.0,78.0,1.0,1600.0,0.0,1.0,0.0,0.0,291.666667,345.0,2.35,1599.0,41.0,38.0,3.0,13.0,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,6.246947,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.200000,81.530691,3.2,3.200000,3.200000,12.667984,3.2,395.799401,3.379747,3.2,3.200000,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,0.0,1.0,72.0,0.0,0.0,0.0,2.0,0.0,0.0,264.571429,277.0,5.65,1620.0,18.0,21.0,0.0,61.0,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,6.246947,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.200000,60.021260,3.2,3.200000,3.200000,16.925359,3.2,1149.151761,6.373706,3.2,3.200000,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,0.0,0.0,86.0,0.0,0.0,0.0,2.0,0.0,0.0,250.000000,269.0,6.50,1073.0,2.0,81.0,4.0,12.0,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,6.246947,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.200000,14.035815,3.2,3.200000,3.200000,23.676270,3.2,67.058080,3.200000,3.2,3.200000,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,0.0,0.0,58.0,1.0,500.0,0.0,2.0,0.0,0.0,200.000000,538.0,11.26,2392.0,13.0,47.0,14.0,25.0,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,6.246947,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,3.200000,36.754655,3.2,3.200000,3.200000,4.925806,3.2,205.470611,3.200000,3.2,3.200000,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,3.792897,3.2,3.2,2.000000
4,0.0,0.0,76.0,1.0,800.0,0.0,3.0,0.0,0.0,380.952381,372.0,6.43,1555.0,9.0,37.0,36.0,7.0,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,4.350069,6.246947,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.200000,946.005892,3.2,3.200000,3.200000,6.572539,3.2,1089.321448,3.200000,3.2,3.200000,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000
5,0.0,0.0,81.0,0.0,0.0,0.0,2.0,0.0,0.0,325.000000,343.0,0.42,5820.0,2.0,81.0,1.0,15.0,3.1,46.258811,9603.296839,11.080007,23.300894,9.765625,4.350069,6.246947,6.671479,3.200000,32.800163,6.137524,3.200000,40.928415,8.294132,3.200000,320.197878,3.200000,15.616715,3.200000,22.958143,3.2,3.200000,3.200000,10.204339,3.2,51.848895,3.200000,3.2,3.200000,3.2,3.2,6.522719,3.20000,3.200000,3.2,27.095784,2143.610593,317.600056,10.880264,29.219451,4.593708,3.2,3.2,2.000000
6,0.0,0.0,78.0,1.0,1000.0,1.0,2.0,0.0,0.0,333.333333,187.0,14.31,358.0,53.0,13.0,5.0,29.0,3.8,234.392214,12086.725456,20.266175,24.842279,9.765625,4.850196,6.246947,4.283046,3.200000,27.138820,3.617591,3.600200,66.422824,6.421678,3.200000,435.097287,3.200000,10.386990,3.200000,77.521431,3.2,3.799654,3.200000,4.925806,3.2,381.310620,10.028918,3.2,3.200000,3.2,3.2,8.902900,3.20000,129.276818,3.2,177.115594,2578.088199,317.652448,12.177016,33.782347,4.216050,3.2,3.2,2.000000
7,0.0,1.0,76.0,1.0,100.0,0.0,3.0,1.0,0.0,285.714286,458.0,4.06,981.0,4.0,76.0,0.0,20.0,1.2,51.976464,3060.022247,9.765625,10.530582,9.765625,4.610201,6.246947,14.628145,3.200000,35.651712,3.200000,3.200000,59.591984,9.538384

In [21]:
# 今回の機械学習に使うライブラリーはsklearnのRF解析
# sklearnが機械学習モデルを適応できるようにデータをnumpyのarrayに変換する
A = AE_FILD_idf_syn.values #.valuesはnumpy arrayを出力する　

# 後で、乗算と除算ができるように、すべての値を浮動小数点に変換する
A = A.astype('float64')
print(A)

[[ 0.          0.         78.         ...  3.2         3.2
   4.22195768]
 [ 0.          1.         72.         ...  3.2         3.2
   2.        ]
 [ 0.          0.         86.         ...  3.2         3.2
   2.        ]
 ...
 [ 1.          1.         66.         ...  3.2        24.44557931
   2.        ]
 [ 0.          1.         67.         ...  3.2        40.90545267
   4.22195768]
 [ 0.          0.         86.         ...  3.2        44.96107768
   4.22195768]]


In [22]:
# 行数と列数を変数として保管する
NUM_ROWS, NUM_COLS = A.shape
print(f"\nこのデータセットの行数：{NUM_COLS}、列数：{NUM_ROWS}")


このデータセットの行数：62、列数：110


In [23]:
# データセットのフィーチャー・ラベル定義

X_all = A[:,1:63].copy()  # X (フィーチャー) ... 1行目以外
y_all = A[:,0].copy()    # y (ラベル) ... 全ての行, 1列目のみ

# Create a copy of X_all and y_all to apply rf analysis to later
X_all_rf = X_all.copy()
y_all_rf = y_all.copy()

print(f"X_all (フィーチャーのみ: 10行まで) \n {X_all[:10,:]}")
print(f"y_all (ラベルのみ) \n {y_all}")

X_all (フィーチャーのみ: 10行まで) 
 [[0.00000000e+00 7.80000000e+01 1.00000000e+00 1.60000000e+03
  0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
  2.91666667e+02 3.45000000e+02 2.35000000e+00 1.59900000e+03
  4.10000000e+01 3.80000000e+01 3.00000000e+00 1.30000000e+01
  3.50000000e+00 6.64773204e+02 1.33137916e+04 6.21768045e+01
  4.04388091e+01 9.76562500e+00 4.45038295e+00 6.24694689e+00
  1.09631327e+01 3.20000000e+00 1.08365991e+01 3.20000000e+00
  3.20000000e+00 3.32659886e+01 3.20000000e+00 3.20000000e+00
  9.54746558e+02 3.20000000e+00 3.47345075e+01 3.20000000e+00
  8.15306905e+01 3.20000000e+00 3.20000000e+00 3.20000000e+00
  1.26679837e+01 3.20000000e+00 3.95799401e+02 3.37974706e+00
  3.20000000e+00 3.20000000e+00 3.20000000e+00 3.20000000e+00
  1.44513353e+01 3.20000000e+00 2.30161662e+01 3.20000000e+00
  3.13439925e+02 2.16094095e+03 1.00987865e+03 2.89063159e+01
  3.09405765e+01 4.48326515e+00 3.20000000e+00 3.20000000e+00
  4.22195768e+00]
 [1.00000000e+00 7.2000000

#### 各パラメーターの重み指定
重みの学習率の初期値についてですが、大きくすると学習が早く進みますが、最適な重みに落ち着かない場合があります。

また、小さすぎると学習の進みが遅くなります。

In [24]:
# フィーチャーの重要性を表すpythonのディクショナリー
# 最初の値は全ての行に1（データの内容を変えないように１で掛け算してもデータは変わらない）で指定しておけば、RFモデルがトレーニングする際に自動で更新してくれる
COL_WEIGHT_DICT = {}

for colname in AE_FILD_idf_syn.columns[1:]: # 最初の行が出力データを表すため、フィーチャーの重みは最初の行以外を考慮する
  i = AE_FILD_idf_syn.columns.get_loc(colname) - 1
  weight = 1
  COL_WEIGHT_DICT[colname] = 1
  print(f"行名{colname}の重みを{weight}に指定しました")
  X_all[:,i] *= weight # そのコラム内のデータをweightで掛け算する

行名sexの重みを1に指定しました
行名ageの重みを1に指定しました
行名smoking-statusの重みを1に指定しました
行名Brinkman-Indexの重みを1に指定しました
行名emphysemaの重みを1に指定しました
行名Baseline-ILDの重みを1に指定しました
行名Collagen-diseaseの重みを1に指定しました
行名IPAFの重みを1に指定しました
行名P/F-ratioの重みを1に指定しました
行名LDHの重みを1に指定しました
行名CRPの重みを1に指定しました
行名KL-6の重みを1に指定しました
行名BAL(neutro-%)の重みを1に指定しました
行名BAL(Lymph-%)の重みを1に指定しました
行名BAL(Eos-%)の重みを1に指定しました
行名BAL(M-%)の重みを1に指定しました
行名BAL(CD4/8)の重みを1に指定しました
行名MMP-1の重みを1に指定しました
行名MMP-7の重みを1に指定しました
行名TGFB1の重みを1に指定しました
行名TGFB2の重みを1に指定しました
行名TGFB3の重みを1に指定しました
行名EGFの重みを1に指定しました
行名FGF-2の重みを1に指定しました
行名EOTAXINの重みを1に指定しました
行名TGF-aの重みを1に指定しました
行名G-CSFの重みを1に指定しました
行名Flt-3Lの重みを1に指定しました
行名GM-CSFの重みを1に指定しました
行名FRACTALKINEの重みを1に指定しました
行名IFN-a2の重みを1に指定しました
行名IFN-gの重みを1に指定しました
行名GROの重みを1に指定しました
行名IL-10の重みを1に指定しました
行名MCP-3の重みを1に指定しました
行名IL-12P40の重みを1に指定しました
行名MDCの重みを1に指定しました
行名IL-12P70の重みを1に指定しました
行名IL-13の重みを1に指定しました
行名IL-15の重みを1に指定しました
行名sCD40Lの重みを1に指定しました
行名IL-17の重みを1に指定しました
行名IL-1RAの重みを1に指定しました
行名IL-1aの重みを1に指定しました
行名IL-9の重みを1に指定しました
行名IL-1bの重みを1に指定しました
行名IL-2

In [25]:
# トレーニングデータとテストデータに入力データを分ける
# トレーニングデータ内のどのデータをどちらのデータセットに振り分けるかの指定をnpのrandom.permutationにより、毎回機械学習を行う際にランダムで決めてもらう
indices = np.random.permutation(len(y_all)) # ラベルの行のindexを記憶している変数
print(f"indices: {indices}")

# X_allとy_allに上記でnpが主力したランダムindexでデータセットの内容をシャッフルする （注：X_allとy_allは必ず同じindexでシャッフルしなくてはいけない）
X_all = X_all[indices]
y_all = y_all[indices]

print("置き換わったフィーチャーリスト：\n", X_all[:10,:])
print("置き換わったラベルリスト：\n", y_all)

indices: [108  61   6  32  75  38  90  20 103  27  81   4  92  33 102  23  11  31
  19  89  95  87  14  74   1   9  93  18  65  12  69   2  48  76  68 101
  78  79 105  47  43  49   5  57   3  82  91  21  13  64 107   0  24  16
  39  98  17  54  58  59  86  56 109  97  60  22  36  15  63  28  94  71
  85  45  26   7  34  50 104  51  66  42  88  35  70  83  53  73  96  72
  80  77  29  41  84  55  10 100  44  67  40  30  25  37   8  46 106  62
  52  99]
置き換わったフィーチャーリスト：
 [[1.00000000e+00 6.70000000e+01 0.00000000e+00 1.79700000e+03
  1.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
  3.13045547e+02 1.12100000e+03 5.84754899e+00 4.80000000e+01
  1.30000000e+01 2.50000000e+01 3.20000000e+01 2.10000000e+01
  2.81262626e+00 3.10260441e+03 4.51629762e+04 7.15202015e+01
  1.13425955e+02 1.18891103e+01 6.18527398e+00 1.79437866e+01
  3.83188592e+01 3.20000000e+00 2.34615997e+02 6.69102539e+00
  3.91172468e+00 3.51874575e+00 3.52147698e+00 2.20446273e+01
  5.23341676e+02 8.28975582e+

#### トレーニングデータセットとテストデータの振り分けの割合を決める

In [26]:
# トレニーングデータとテストデータの振り分け
# 割合：トレーニング（8割）・テスト（2割）
NUM_ROWS = X_all.shape[0] # 列数
TEST_PERCENT = 0.2 # TEST ＋ TRAINING は必ず1にならなければいけない
TEST_SIZE = int(TEST_PERCENT * NUM_ROWS) 

X_test = X_all[:TEST_SIZE] # 0からTEST＿SIZEまでのデータ
y_test = y_all[:TEST_SIZE]

X_train = X_all[TEST_SIZE:] # TEST_SIZE以降のデータ
y_train = y_all[TEST_SIZE:]

print(f"トレーニング列数：{len(y_train)}、テスト列数：{len(y_test)}")

トレーニング列数：88、テスト列数：22


#### 実際のRF解析モデルの定義

In [27]:
# RF解析に使うsklearnのライブラリーの導入
from sklearn import tree # 決定木モデル
from sklearn import ensemble # RF解析用のモデル
from sklearn.model_selection import cross_val_score # RF解析の最適解層をcross validationを使い見つけるためのモデル

In [28]:
# モデルの構築・モデルのトレーニング
# RF モデルの構築
# cross_valを使い、最適層を決めるんですが、事前に何階層を使えばいいかわからないため、適当に6階層決定木42と指定する
best_depth = 6
best_num_trees = 42
rforest_model = ensemble.RandomForestClassifier(max_depth=best_depth, n_estimators=best_num_trees) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルの初期トレーニング
rforest_model.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{best_depth}層と{best_num_trees}決定木で作成しました")

RFモデルを6層と42決定木で作成しました


In [29]:
# 初期トレーニング後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"正確性は：{num_correct/total * 100}％でした")

予測ラベル：[1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
実際ラベル: [0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
正確性は：54.54545454545454％でした


In [30]:
#  ラベル毎に予測ラベルが実際のラベルと一致してるか確認する関数
def compare_labels(predicted_labels, actual_labels):
    NUM_LABELS = len(predicted_labels)
    num_correct = 0
    
    for i in range(NUM_LABELS): 
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {p} {a}   {result}") 

    print(f"\n正確性は：{num_correct/NUM_LABELS * 100}％でした")
    return num_correct

#### 色々なハイパーパラメータチューニング法

##### cross-validation

In [31]:
#@title
# cross validationを使い、階層や決定木の数をチューニングしてみる（ハイパーパラメーター）
# 注：過適合/過剰適合のバランスを見極めながらハイパーパラメーターチューニングを行う
best_accuracy = 0
best_depth = 1
best_num_trees = 42

for ntrees in range(50,450,100):  # 決定木50から350の間（ステップ数：100)
    for d in range(1,20): # 階層1から20の間
        rforest_model = ensemble.RandomForestClassifier(max_depth=d, 
                                                        n_estimators=ntrees)
        cv_scores = cross_val_score( rforest_model, X_train, y_train, cv=5 ) # 5 はトレーニング・テストの割合が8割・2割ということを表す
        average_cv_accuracy = cv_scores.mean()  # cvスコアの平均値cv=5はスコアを五つ出すため、その平均値を見る
        if average_cv_accuracy >= best_accuracy: # cvスコアの平均値正確性が現在の最適正確性を上回った場合、現在のcvスコア平均値を出した決定木数・階層を最適とする
            best_accuracy = average_cv_accuracy
            best_depth = d
            best_num_trees = ntrees
        print(f"階層: {d:2d} 決定木: {ntrees:3d} cv スコア平均値: {average_cv_accuracy:7.4f}")

print()
print(f"最適層: {best_depth}, 最適決定木数： {best_num_trees}, 最高予測精度：{best_accuracy}")

階層:  1 決定木:  50 cv スコア平均値:  0.6477
階層:  2 決定木:  50 cv スコア平均値:  0.6810
階層:  3 決定木:  50 cv スコア平均値:  0.7150
階層:  4 決定木:  50 cv スコア平均値:  0.6928
階層:  5 決定木:  50 cv スコア平均値:  0.6941
階層:  6 決定木:  50 cv スコア平均値:  0.7039
階層:  7 決定木:  50 cv スコア平均値:  0.6824
階層:  8 決定木:  50 cv スコア平均値:  0.6359
階層:  9 決定木:  50 cv スコア平均値:  0.7163
階層: 10 決定木:  50 cv スコア平均値:  0.7288
階層: 11 決定木:  50 cv スコア平均値:  0.6810
階層: 12 決定木:  50 cv スコア平均値:  0.7275
階層: 13 決定木:  50 cv スコア平均値:  0.6477
階層: 14 決定木:  50 cv スコア平均値:  0.6824
階層: 15 決定木:  50 cv スコア平均値:  0.6686
階層: 16 決定木:  50 cv スコア平均値:  0.6699
階層: 17 決定木:  50 cv スコア平均値:  0.6928
階層: 18 決定木:  50 cv スコア平均値:  0.7039
階層: 19 決定木:  50 cv スコア平均値:  0.6712
階層:  1 決定木: 150 cv スコア平均値:  0.6595
階層:  2 決定木: 150 cv スコア平均値:  0.6359
階層:  3 決定木: 150 cv スコア平均値:  0.6706
階層:  4 決定木: 150 cv スコア平均値:  0.6588
階層:  5 決定木: 150 cv スコア平均値:  0.7039
階層:  6 決定木: 150 cv スコア平均値:  0.7046
階層:  7 決定木: 150 cv スコア平均値:  0.6935
階層:  8 決定木: 150 cv スコア平均値:  0.7039
階層:  9 決定木: 150 cv スコア平均値:  0.7039
階層: 10 決定木: 150 cv ス

In [32]:
#@title
# 上記で見つけた最適層・最適決定木数をもとにモデルの構築・モデルのトレーニング
rforest_model_cv = ensemble.RandomForestClassifier(max_depth=best_depth, n_estimators=best_num_trees) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルを再度トレーニングする
rforest_model_cv.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{best_depth}層と{best_num_trees}決定木で作成しました")

RFモデルを11層と250決定木で作成しました


###### クロスバリデーションを終えた後のモデルのパフォーマンス検証

In [33]:
#@title
# 再度トレーニングした後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model_cv.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
compare_labels(predicted_labels, actual_labels)

予測ラベル：[0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
実際ラベル: [0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
row   0 : 0 0   
row   1 : 1 1   
row   2 : 0 0   
row   3 : 0 0   
row   4 : 1 1   
row   5 : 1 0   incorrect
row   6 : 0 1   incorrect
row   7 : 0 0   
row   8 : 1 1   
row   9 : 0 1   incorrect
row  10 : 0 1   incorrect
row  11 : 0 0   
row  12 : 0 1   incorrect
row  13 : 0 0   
row  14 : 0 1   incorrect
row  15 : 0 0   
row  16 : 0 0   
row  17 : 0 0   
row  18 : 0 0   
row  19 : 1 1   
row  20 : 0 0   
row  21 : 1 0   incorrect

正確性は：68.18181818181817％でした


15

##### Optuna

In [34]:
!pip install optuna
import optuna
def objective(trial):
  n_estimators = trial.suggest_int('n_estimators', 2, 20)
  max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
  rforest_model_optuna = ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
  return cross_val_score(rforest_model_optuna, X_train, y_train, n_jobs=-1, cv=5).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))

print("Best hyperparameters: {}".format(trial.params))

[I 2022-02-21 05:52:28,976] A new study created in memory with name: no-name-eae8450d-fd22-4228-99dc-3e75f9493e5f
[I 2022-02-21 05:52:30,709] Trial 0 finished with value: 0.5915032679738562 and parameters: {'n_estimators': 16, 'max_depth': 16.25988377943964}. Best is trial 0 with value: 0.5915032679738562.
[I 2022-02-21 05:52:30,818] Trial 1 finished with value: 0.6470588235294118 and parameters: {'n_estimators': 5, 'max_depth': 1.4287619746907252}. Best is trial 1 with value: 0.6470588235294118.
[I 2022-02-21 05:52:31,101] Trial 2 finished with value: 0.6934640522875817 and parameters: {'n_estimators': 20, 'max_depth': 16.775476207521812}. Best is trial 2 with value: 0.6934640522875817.
[I 2022-02-21 05:52:31,383] Trial 3 finished with value: 0.6477124183006537 and parameters: {'n_estimators': 18, 'max_depth': 24.605837958047413}. Best is trial 2 with value: 0.6934640522875817.
[I 2022-02-21 05:52:31,535] Trial 4 finished with value: 0.661437908496732 and parameters: {'n_estimators': 

Accuracy: 0.7385620915032679
Best hyperparameters: {'n_estimators': 15, 'max_depth': 3.889142958844008}


In [35]:
# 上記で見つけた最適層・最適決定木数をもとにモデルの構築・モデルのトレーニング

n_estimators = trial.params['n_estimators']
max_depth = trial.params['max_depth']
rforest_model_post_optuna = ensemble.RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルを再度トレーニングする
rforest_model_post_optuna.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{max_depth}層と{n_estimators}決定木で作成しました")

RFモデルを3.889142958844008層と15決定木で作成しました


###### Optunaクロスバリデーションを終えた後のモデルのパフォーマンス検証

In [36]:
# 再度トレーニングした後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model_post_optuna.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
compare_labels(predicted_labels, actual_labels)

予測ラベル：[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
実際ラベル: [0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
row   0 : 0 0   
row   1 : 0 1   incorrect
row   2 : 0 0   
row   3 : 0 0   
row   4 : 1 1   
row   5 : 1 0   incorrect
row   6 : 1 1   
row   7 : 0 0   
row   8 : 0 1   incorrect
row   9 : 0 1   incorrect
row  10 : 1 1   
row  11 : 0 0   
row  12 : 1 1   
row  13 : 0 0   
row  14 : 0 1   incorrect
row  15 : 0 0   
row  16 : 0 0   
row  17 : 0 0   
row  18 : 0 0   
row  19 : 1 1   
row  20 : 1 0   incorrect
row  21 : 1 0   incorrect

正確性は：68.18181818181817％でした


15

###### Optunaクロスバリデーションの結果の表現

In [37]:
optuna.visualization.plot_optimization_history(study)

In [38]:
optuna.visualization.plot_slice(study)

#### 学習モデルにより出力されたフィーチャーの重要度のランキング化

##### クロスバリデーション

In [39]:
# フィーチャーの重要度を出力するコード
feature_importances = rforest_model_cv.feature_importances_

feature_importances_dict = {}
for i, importance in enumerate(feature_importances):
  perc = importance * 100
  feature_importances_dict[AE_FILD_idf_syn.columns[i+1]] = perc # 一行目は出力行のため、考慮しない

sorted_dict = {}
sorted_keys = sorted(feature_importances_dict, key=feature_importances_dict.get)
for w in sorted_keys:
    sorted_dict[w] =  feature_importances_dict[w]
  
for keys in sorted_dict.keys():
  print(f"フィーチャー名：{keys:>12s}は出力データの結果に{sorted_dict[keys]:>7.2f}%関与しています")

フィーチャー名：    IL-12P70は出力データの結果に   0.00%関与しています
フィーチャー名：       IL-17は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-9は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-2は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-3は出力データの結果に   0.00%関与しています
フィーチャー名：        TNFbは出力データの結果に   0.00%関与しています
フィーチャー名：         sexは出力データの結果に   0.17%関与しています
フィーチャー名：      IL-36βは出力データの結果に   0.21%関与しています
フィーチャー名：        IPAFは出力データの結果に   0.21%関与しています
フィーチャー名：   emphysemaは出力データの結果に   0.25%関与しています
フィーチャー名：Collagen-diseaseは出力データの結果に   0.39%関与しています
フィーチャー名：       TGF-aは出力データの結果に   0.42%関与しています
フィーチャー名：       IFN-gは出力データの結果に   0.46%関与しています
フィーチャー名：smoking-statusは出力データの結果に   0.57%関与しています
フィーチャー名：Baseline-ILDは出力データの結果に   0.62%関与しています
フィーチャー名：        IL-7は出力データの結果に   0.80%関与しています
フィーチャー名：        IL-5は出力データの結果に   0.84%関与しています
フィーチャー名：    IL-12P40は出力データの結果に   0.96%関与しています
フィーチャー名：  BAL(Eos-%)は出力データの結果に   1.05%関与しています
フィーチャー名：        TNFaは出力データの結果に   1.09%関与しています
フィーチャー名：       TGFB3は出力データの結果に   1.12%関与しています
フィーチャー名：        VEGFは出力データの結

##### Optunaクロスバリデーション

In [40]:
# フィーチャーの重要度を出力するコード
feature_importances = rforest_model_post_optuna.feature_importances_

feature_importances_dict = {}
for i, importance in enumerate(feature_importances):
  perc = importance * 100
  feature_importances_dict[AE_FILD_idf_syn.columns[i+1]] = perc # 一行目は出力行のため、考慮しない

sorted_dict = {}
sorted_keys = sorted(feature_importances_dict, key=feature_importances_dict.get)
for w in sorted_keys:
    sorted_dict[w] =  feature_importances_dict[w]
  
for keys in sorted_dict.keys():
  print(f"フィーチャー名：{keys:>12s}は出力データの結果に{sorted_dict[keys]:>7.2f}%関与しています")

フィーチャー名：         sexは出力データの結果に   0.00%関与しています
フィーチャー名：smoking-statusは出力データの結果に   0.00%関与しています
フィーチャー名：Collagen-diseaseは出力データの結果に   0.00%関与しています
フィーチャー名：        IPAFは出力データの結果に   0.00%関与しています
フィーチャー名：         LDHは出力データの結果に   0.00%関与しています
フィーチャー名：  BAL(Eos-%)は出力データの結果に   0.00%関与しています
フィーチャー名：         EGFは出力データの結果に   0.00%関与しています
フィーチャー名：       TGF-aは出力データの結果に   0.00%関与しています
フィーチャー名：      IFN-a2は出力データの結果に   0.00%関与しています
フィーチャー名：    IL-12P70は出力データの結果に   0.00%関与しています
フィーチャー名：       IL-17は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-9は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-2は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-3は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-5は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-7は出力データの結果に   0.00%関与しています
フィーチャー名：      MIP-1aは出力データの結果に   0.00%関与しています
フィーチャー名：        TNFbは出力データの結果に   0.00%関与しています
フィーチャー名：      IL-36βは出力データの結果に   0.00%関与しています
フィーチャー名：Baseline-ILDは出力データの結果に   0.38%関与しています
フィーチャー名：   emphysemaは出力データの結果に   0.40%関与しています
フィーチャー名： FRACTALKINEは出力データの結

In [46]:
# 上記のトップ10因子を基に新しい機械学習モデルを組む
top_ten_params = list(sorted_dict.keys())[-10:]
top_ten_params.append('90day-mortality')
print(top_ten_params)

#　最後に使用したPandasデータフレームをコピーを作成する
AE_FILD_idf_syn_c = AE_FILD_idf_syn.copy()

# 新しいデータフレームから重要因子以外を取り除く
AE_FILD_idf_syn_c = AE_FILD_idf_syn_c.drop(columns=[col for col in AE_FILD_idf_syn.columns if col not in top_ten_params])

# sklearnが機械学習モデルを適応できるようにデータをnumpyのarrayに変換する
A = AE_FILD_idf_syn_c.values #.valuesはnumpy arrayを出力する

# 後で、乗算と除算ができるように、すべての値を浮動小数点に変換する
A = A.astype('float64')

# データセットのフィーチャー・ラベル定義
X_all = A[:,1:12].copy()  # X (フィーチャー) ... 1行目以外
y_all = A[:,0].copy()    # y (ラベル) ... 全ての行, 1列目のみ

# フィーチャーの重要性を表すpythonのディクショナリー
# 最初の値は全ての行に1（データの内容を変えないように１で掛け算してもデータは変わらない）で指定しておけば、RFモデルがトレーニングする際に自動で更新してくれる
COL_WEIGHT_DICT = {}

for colname in AE_FILD_idf_syn_c.columns[1:]: # 最初の行が出力データを表すため、フィーチャーの重みは最初の行以外を考慮する
  i = AE_FILD_idf_syn_c.columns.get_loc(colname) - 1
  weight = 1
  COL_WEIGHT_DICT[colname] = 1
  print(f"行名{colname}の重みを{weight}に指定しました")
  X_all[:,i] *= weight # そのコラム内のデータをweightで掛け算する

# トレーニングデータとテストデータに入力データを分ける
# トレーニングデータ内のどのデータをどちらのデータセットに振り分けるかの指定をnpのrandom.permutationにより、毎回機械学習を行う際にランダムで決めてもらう
indices = np.random.permutation(len(y_all)) # ラベルの行のindexを記憶している変数
print(f"indices: {indices}")

# X_allとy_allに上記でnpが主力したランダムindexでデータセットの内容をシャッフルする （注：X_allとy_allは必ず同じindexでシャッフルしなくてはいけない）
X_all = X_all[indices]
y_all = y_all[indices]

X_test = X_all[:TEST_SIZE] # 0からTEST＿SIZEまでのデータ
y_test = y_all[:TEST_SIZE]

X_train = X_all[TEST_SIZE:] # TEST_SIZE以降のデータ
y_train = y_all[TEST_SIZE:]

# Optunaで新しいRF解析モデルの指定
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))

print("Best hyperparameters: {}".format(trial.params))

# 上記で見つけた最適層・最適決定木数をもとにモデルの構築・モデルのトレーニング

n_estimators = trial.params['n_estimators']
max_depth = trial.params['max_depth']
rforest_model_post_optuna = ensemble.RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルを再度トレーニングする
rforest_model_post_optuna.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{max_depth}層と{n_estimators}決定木で作成しました")

# 再度トレーニングした後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model_post_optuna.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
compare_labels(predicted_labels, actual_labels)

# フィーチャーの重要度を出力するコード
feature_importances = rforest_model_post_optuna.feature_importances_

feature_importances_dict = {}
for i, importance in enumerate(feature_importances):
  perc = importance * 100
  feature_importances_dict[AE_FILD_idf_syn_c.columns[i+1]] = perc

sorted_dict = {}
sorted_keys = sorted(feature_importances_dict, key=feature_importances_dict.get)
for w in sorted_keys:
    sorted_dict[w] =  feature_importances_dict[w]
  
for keys in sorted_dict.keys():
  print(f"フィーチャー名：{keys:>12s}は出力データの結果に{sorted_dict[keys]:>7.2f}%関与しています")

[I 2022-02-21 06:02:15,927] A new study created in memory with name: no-name-f971596d-be2e-4157-96ea-19cd60e3333c


['MMP-7', 'G-CSF', 'P/F-ratio', 'IL-10', 'EOTAXIN', 'IL-6', 'MMP-1', 'BAL(neutro-%)', 'IL-8', 'IL-4', '90day-mortality']
行名P/F-ratioの重みを1に指定しました
行名BAL(neutro-%)の重みを1に指定しました
行名MMP-1の重みを1に指定しました
行名MMP-7の重みを1に指定しました
行名EOTAXINの重みを1に指定しました
行名G-CSFの重みを1に指定しました
行名IL-10の重みを1に指定しました
行名IL-4の重みを1に指定しました
行名IL-6の重みを1に指定しました
行名IL-8の重みを1に指定しました
indices: [ 34  22  95   5  46  24  86 102  38  11  14  43   8  61 100  41  62  77
   9  47  96  64  78  44  28  85  20  49  82  12  32  79 106  57  35  68
  84  36  99  55  66  93  13  91  45  25   2 105  31   7  40   1  16  67
  90  60  59  76   3  72  88  39  75  50 109  21   6  51  30  48  52  15
  83  54 104  74  80  87  23  92  56  10 107   0  37  42  63  27  33  69
  29  26 108  70  58  89  18  97  19  98  71  81  94  53 103 101  65  17
   4  73]


[I 2022-02-21 06:02:18,688] Trial 0 finished with value: 0.761437908496732 and parameters: {'n_estimators': 19, 'max_depth': 6.697001429297231}. Best is trial 0 with value: 0.761437908496732.
[I 2022-02-21 06:02:18,804] Trial 1 finished with value: 0.7261437908496732 and parameters: {'n_estimators': 6, 'max_depth': 8.331543159292377}. Best is trial 0 with value: 0.761437908496732.
[I 2022-02-21 06:02:18,885] Trial 2 finished with value: 0.7137254901960784 and parameters: {'n_estimators': 3, 'max_depth': 14.912457243480333}. Best is trial 0 with value: 0.761437908496732.
[I 2022-02-21 06:02:19,223] Trial 3 finished with value: 0.7823529411764707 and parameters: {'n_estimators': 19, 'max_depth': 9.229091787746297}. Best is trial 3 with value: 0.7823529411764707.
[I 2022-02-21 06:02:19,595] Trial 4 finished with value: 0.7163398692810456 and parameters: {'n_estimators': 18, 'max_depth': 3.409499016333691}. Best is trial 3 with value: 0.7823529411764707.
[I 2022-02-21 06:02:19,838] Trial 5

Accuracy: 0.8274509803921568
Best hyperparameters: {'n_estimators': 13, 'max_depth': 6.245860891421765}
RFモデルを6.245860891421765層と13決定木で作成しました
予測ラベル：[0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1.]
実際ラベル: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.]
row   0 : 0 0   
row   1 : 0 0   
row   2 : 1 0   incorrect
row   3 : 0 0   
row   4 : 0 0   
row   5 : 0 0   
row   6 : 0 0   
row   7 : 0 1   incorrect
row   8 : 1 0   incorrect
row   9 : 1 0   incorrect
row  10 : 0 1   incorrect
row  11 : 0 0   
row  12 : 0 0   
row  13 : 1 1   
row  14 : 1 0   incorrect
row  15 : 1 0   incorrect
row  16 : 1 0   incorrect
row  17 : 0 0   
row  18 : 1 1   
row  19 : 0 0   
row  20 : 1 1   
row  21 : 1 1   

正確性は：63.63636363636363％でした
フィーチャー名：       G-CSFは出力データの結果に   5.11%関与しています
フィーチャー名：     EOTAXINは出力データの結果に   5.85%関与しています
フィーチャー名：BAL(neutro-%)は出力データの結果に   8.33%関与しています
フィーチャー名：        IL-6は出力データの結果に   8.59%関与しています
フィーチャー名：   P/F-ratioは出力データの結果に   8.64%関与しています
フィーチ

## 他に試した模擬データ作成方法。上手くいかなかったため、考慮しなくても良い

In [ ]:
# ! pip install ydata_synthetic
# # 機械学習モデルがデータを学習する為に使うトランスフォメーション層の導入
# from ydata_synthetic.preprocessing import *

# train_data = AE_FILD_df.loc[AE_FILD_df['90day mortality survival:0 death:1']==1].copy()

# data = transformations(train_data)

# print("Dataset info: Number of records - {} Number of variables - {}".format(train_data.shape[0], train_data.shape[1]))

In [ ]:
# from sklearn.neighbors import KernelDensity
# from sklearn.model_selection import GridSearchCV

# # ガウス法に基づきまずデータセットの密度をnumpyで表すためbandwithの設定
# # このbandwithの値に基づきデータ全体のパターンを機械学習モデルが予測しようとするため、機械学習モデルが勝手に最適bandwithを見つけてくれるものの、初期設定は重要
# bandwith_params = {'bandwidth': np.arange(0.01,1,0.05)} # [0.01 から１]の間でステップ0.05で値を作成
# # モデルをスキャンして最適パラメーターを見つける機械学習モデル（模擬データを作成するのに最適）
# grid_search = GridSearchCV(KernelDensity(), bandwith_params)
# # Aは従来の患者データ。fit()することで上記の機械学習モデルをトレーニングしてデータ内のパターンを学習してもらう
# grid_search.fit(A)
# kde = grid_search.best_estimator_

# # rand_stateの定義（rand_stateは新しいデータを作成する際のseedの定義）
# rand_state = 11
# # トレーニングしたモデルを使い新しい症例を20例作成してもらう
# new_data = kde.sample(20, random_state = rand_state)

# # 新しい症例を元のデータに加る
# A_updated = np.concatenate((A, new_data), axis=0)